# Training and Registering Models on Amazon SageMaker Pipelines Integrated with Presto


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

---

Customers can use SageMaker Pipelines to build scalable machine learning pipelines that preprocess data and train machine learning models. With SageMaker Pipelines, customers have a toolkit for every part of the machine learning lifecycle that provides deep customizations and tuning options to fit every organization. Customers have the freedom to customize SageMaker Pipelines to specific use cases, but also to create generic machine learning pipelines that can be reused across different use cases.

From a birds-eye view a machine learning pipeline usually consists of 3 general steps: a preprocess step where the data is transformed, a training step where a machine learning model is trained, and an evaluation step which tests the performance of the trained model. If the model is performing according to the objective metric you’re optimizing for, then that becomes a candidate model for deployment to one or more environments. These candidate models should be registered into SageMaker Model Registry to catalog and store key metadata for that model version.

--- 

These steps have a lot of commonalities, even across different machine learning use cases. Customers that want to create training pipelines that can be re-used in an organization can use SageMaker Pipelines to create parameterized, generic training pipelines. Parameters allow customers to identify specific parameters that can be passed into the pipeline during pipeline execution without having to directly change the pipeline code itself. 

**This notebook** demonstrates how SageMaker Pipelines can be used to create a generic binary classification machine learning pipeline using Random Forest that's reusable across teams, machine learning use cases and even customers in a SaaS system. 


### SageMaker Pipelines
Amazon SageMaker Pipelines is a purpose-built, easy-to-use CI/CD service for machine learning. With SageMaker Pipelines, customers can create machine learning workflows with an easy-to-use Python SDK, and then visualize and manage workflows using Amazon SageMaker Studio.


#### SageMaker Pipeline steps and parameters
SageMaker pipelines works on the concept of steps. The order steps are executed in is inferred from the dependencies each step has. If a step has a dependency on the output from a previous step, it's not executed until after that step has completed successfully.

SageMaker Pipeline Parameters are input parameters specified when triggering a pipeline execution. They need to be explicitly defined when creating the pipeline and contain default values.
(https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html).

#### SageMaker Pipeline DAG

When creating a SageMaker Pipeline, SageMaker creates a Direct Acyclic Graph, DAG, that customers can visualize in Amazon SageMaker Studio. The DAG can be used to track pipeline executions, outputs and metrics. In this notebook, a SageMaker Pipeline with the following DAG is created:

## Predict customer orders with Random Forest Classifier

### Data

This notebook uses PrestoDB to extract `tpc-h` data from the `tpc-h connector`, and includes the data extraction, preprocesing, as well as the splitting of data into train, test, and validation datases as a part of the preprocessing step of this sagemaker pipeline. 

***To configure PrestoDB within your EC2 instance view***: [PrestoDB EC2 Connection](https://normanlimxk.com/2020/09/15/creating-a-presto-cluster-on-ec2/)


### Overview 
**Disclaimer** This notebook was created using [Amazon SageMaker Studio](https://aws.amazon.com/sagemaker/studio/) and the `Python3(DataScience) kernel`. SageMaker Studio is required for the visualizations of the DAG and model metrics to work.

The purpose of this notebook is to demonstrate how SageMaker Pipelines can be used to create a generic Scikit-Learn training pipeline that preprocesses, trains, tunes, evaluates and registers new machine learning models with the SageMaker model registry, that is reusable across teams, customers and use cases. All scripts to preprocess the data and evaluate the trained model have been prepared in advance and are available here: 

---

This model is a binary classification model creating using the scikit-learn `RandomForestClassifier`. It categorizes input data into high value/low value order classes. 

Training data: the training data for this model is available via PrestoDB tables and is read into Pandas through the PrestoDB Python client. This data is then read into an Apache Spark dataframe (although the model training happens only using the data in the Pandas dataframe).

* Data is read using queries from PrestoDB and any feature engineering required is done as part of the query itself.

* Note that ingestion of raw data into PrestoDB tables is outside the scope of this project and it is assumed that for the purpose of model training the data can simply be queried from PrestoDB tables.


In [2]:
!pip install -U sagemaker --quiet # Ensure correct version of SageMaker is installed


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
## Install the necessary boto3 and sagemaker libraries to initialize session
import json
import boto3
import sagemaker
from utils import *
import sagemaker.session
from typing import Dict, List, Optional, Tuple, Union
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
## set the logger to track all of the logs as this pipeline runs
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### Load the Config.yml file that contains information that is used across this pipeline

In [5]:
config = load_config('config.yml')
logger.info(json.dumps(config, indent=2))

[2024-02-22 18:32:23,795] p145 {2294058105.py:2} INFO - {
  "general": {
    "pipeline_name": "mlops-pipeline-presto",
    "prefix": "mlops-pipeline-model",
    "model_group": "mlops-presto-model-registry",
    "model_name": "mlops-presto",
    "endpoint_config_name": "random-forest-classifier",
    "endpoint_name": "mlops-realtime-ep",
    "transform_step_name": "mlops-RandomForestTransform",
    "transform_pipeline_name": "sagemaker-batch-inference"
  },
  "aws": {
    "region": "us-east-1",
    "sagemaker_execution_role": "arn:aws:iam::597703351594:role/service-role/AmazonSageMaker-ExecutionRole-20230918T115140"
  },
  "pipeline_parameters": {
    "presto_host": "3.93.186.209",
    "port_parameter": "8080",
    "user_parameter": "ec2-user",
    "execution_display_name": "executeMLOPs-pipeline"
  },
  "training_params": {
    "training_target": "high_value_order",
    "training_features": [
      "total_extended_price",
      "avg_discount",
      "total_quantity"
    ]
  },
  "train

In [6]:
## initialize the sagemaker session, region, role bucket and pipeline session
session = sagemaker.session.Session()
pipeline_session = PipelineSession()
role = config['aws']['sagemaker_execution_role']
session_bucket = session.default_bucket()

logger.info(f"the pipeline bucket being used for this pipeline execution -> {session_bucket}")
logger.info(f"the sagemaker execution role being used across this pipeline -> {role}")

[2024-02-22 18:32:24,408] p145 {2500438050.py:7} INFO - the pipeline bucket being used for this pipeline execution -> sagemaker-us-east-1-597703351594
[2024-02-22 18:32:24,408] p145 {2500438050.py:8} INFO - the sagemaker execution role being used across this pipeline -> arn:aws:iam::597703351594:role/service-role/AmazonSageMaker-ExecutionRole-20230918T115140


In [7]:
prefix = config['general']['prefix']  # Prefix to S3 artifacts
pipeline_name = config['general']['pipeline_name']  # SageMaker Pipeline name
order_model_group = config['general']['model_group']

logger.info(f"the prefix for the pipeline name -> {prefix}, pipeline name of this execution -> {pipeline_name}, model group name -> {order_model_group}")

[2024-02-22 18:32:24,417] p145 {3100703845.py:5} INFO - the prefix for the pipeline name -> mlops-pipeline-model, pipeline name of this execution -> mlops-pipeline-presto, model group name -> mlops-presto-model-registry


### Set parameters that are used throughout the training pipeline

In [8]:
import json
from sagemaker.workflow.parameters import ParameterString, ParameterInteger, ParameterFloat

# Convert your list to a JSON string
training_features_str = json.dumps(config['training_params']['training_features'])
logger.info(f"the training features being used for this pipeline --> {training_features_str}")

# Define new pipeline parameters
host_parameter = ParameterString(name="HostParameter", default_value=config['pipeline_parameters']['presto_host'])
port_parameter = ParameterString(name="PortParameter", default_value=config['pipeline_parameters']['port_parameter'])
user_parameter = ParameterString(name="UserParameter", default_value=config['pipeline_parameters']['user_parameter'])
target_parameter = ParameterString(name="Target", default_value=config['training_params']['training_target'])
feature_parameter = ParameterString(name="Feature", default_value=training_features_str)

# training hyperparameters to use hyperparameter parameters
n_estimators_parameter = ParameterInteger(name="NEstimators", default_value=config['training_args']['n_estimators'])
max_depth_parameter = ParameterInteger(name="MaxDepth", default_value=config['training_args']['max_depth'])
min_samples_split_parameter = ParameterInteger(name="MinSamplesSplit", default_value=config['training_args']['min_samples_split'])
max_features_parameter = ParameterString(name="MaxFeatures", default_value=config['training_args']['max_features'])
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

# Log the feature parameter as an array
logger.info(f"the feature parameter being used for training -> {feature_parameter.expr}")
logger.info(f"the host parameter being used from the presto config -> {host_parameter.expr}")
logger.info(f"the port parameter being used from the presto config -> {port_parameter.expr}")
logger.info(f"the user parameter being used from the presto config -> {user_parameter.expr}")

[2024-02-22 18:32:24,494] p145 {971140337.py:6} INFO - the training features being used for this pipeline --> ["total_extended_price", "avg_discount", "total_quantity"]
[2024-02-22 18:32:24,495] p145 {971140337.py:25} INFO - the feature parameter being used for training -> {'Get': 'Parameters.Feature'}
[2024-02-22 18:32:24,496] p145 {971140337.py:26} INFO - the host parameter being used from the presto config -> {'Get': 'Parameters.HostParameter'}
[2024-02-22 18:32:24,497] p145 {971140337.py:27} INFO - the port parameter being used from the presto config -> {'Get': 'Parameters.PortParameter'}
[2024-02-22 18:32:24,498] p145 {971140337.py:28} INFO - the user parameter being used from the presto config -> {'Get': 'Parameters.UserParameter'}


<a id='parameters'></a>

### Pipeline input parameters

Pipeline Parameters are input parameter when triggering a pipeline execution. They need to be explicitly defined when creating the pipeline and contain default values.

Create parameters for the inputs to the pipeline. In this case, parameters will be used for:
- `ModelGroup` - Which registry to register the trained model with.
- `InputData` - S3 URI to pipeline input data.
- `PreprocessScript` - S3 URI to python script to preprocess the data.
- `EvaluateScript` - S3 URI to python script to evaluate the trained model.
- `MaxiumTrainingJobs` - How many training jobs to allow when hyperparameter tuning the model
- `MaxiumParallelTrainingJobs` - How many training jobs to allow in parallel when hyperparameter tuning the model.
- `AccuracyConditionThreshold` - Only register models with the model registry if the have at least this classification accuracy.
- `ProcessingInstanceType` - What EC2 instance type to use for processing.
- `TrainingInstanceType` - What EC2 instance type to use for training.

In [9]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)
# To what Registry to register the model and its versions.
model_registry_package = ParameterString(name="ModelGroup", default_value="default-registry")

# Maximum amount of training jobs to allow in the HP tuning
max_training_jobs = ParameterInteger(name="MaxiumTrainingJobs", default_value=config['input_params']['maxium_training_jobs'])

# Maximum amount of trainingjobs to allow in the HP tuning
max_parallel_training_jobs = ParameterInteger(name="MaxiumParallelTrainingJobs", default_value=config['input_params']['maxium_parallel_training_jobs'])

# Accuracy threshold to decide whether or not to register the model with Model Registry
accuracy_condition_threshold = ParameterFloat(name="AccuracyConditionThreshold", default_value=config['input_params']['accuracy_condition_threshold'])

# What instance type to use for processing.
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value=config['input_params']['processing_instance_type']
)

# What instance type to use for training.
training_instance_type = ParameterString(name="TrainingInstanceType", default_value=config['input_params']['training_instance_type'])

<a id='preprocess'></a>

## Preprocess data step
--- 
In the first step an sklearn processor is created, used in the ProcessingStep. In this step, the preprocess script is read to connect to presto and query data, that is then sent to an Amazon S3 bucket split into train, test and validation datasets. Using these files, this step can then use the data for training the model.

In [22]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.functions import Join
from sagemaker.workflow.execution_variables import ExecutionVariables

# Create SKlearn processor object,
# The object contains information about what instance type to use, the IAM role to use etc.
# A managed processor comes with a preconfigured container, so only specifying version is required.
sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1", role=role, instance_type=processing_instance_type, instance_count=1, 
    tags=config['pipeline_tags']['sklearn_processor_tags']
)
# Use the sklearn_processor in a SageMaker Pipelines ProcessingStep
# Configure the ProcessingStep
step_preprocess_data = ProcessingStep(
    name="Preprocess-Data",
    processor=sklearn_processor,
    inputs=[],  # No static inputs required as data fetching is part of the script
    outputs=[
        ProcessingOutput(
            output_name="train",
            source="/opt/ml/processing/train",
            destination=Join(
                on="/",
                values=[
                    "s3://{}".format(session_bucket),
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "train",
                ],
            ),
        ),
        ProcessingOutput(
            output_name="validation",
            source="/opt/ml/processing/validation",
            destination=Join(
                on="/",
                values=[
                    "s3://{}".format(session_bucket),
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "validation",
                ],
            ),
        ),
        ProcessingOutput(
            output_name="test",
            source="/opt/ml/processing/test",
            destination=Join(
                on="/",
                values=[
                    "s3://{}".format(session_bucket),
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "test",
                ],
            ),
        ),
    ],
    code = config['dir_scripts']['preprocess_script'],
    job_arguments=[
        "--host", host_parameter,
        "--port", port_parameter,
        "--user", user_parameter,
    ]
)

[2024-02-22 18:53:41,196] p145 {utilities.py:422} WARNING - The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is interpreted in pipeline execution time only. As the function needs to evaluate the argument value in SDK compile time, the default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.
[2024-02-22 18:53:41,197] p145 {image_uris.py:581} INFO - Defaulting to only available Python version: py3


<a id='train'></a>

## Train model step
In the second step, the train and validation output from the previous processing step are used to train a model. 

---

We use the SKLearn estimator from SageMaker SDK and the RandomForestClassifier from scikit-learn to train the ML model. The HyperparameterTunerclass is used for running automatic model tuning to determine the set of hyperparameters that provide the best performance (maximize the AUC metric).

In [11]:
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
from sklearn.metrics import roc_auc_score
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter, IntegerParameter, CategoricalParameter
from sagemaker.workflow.steps import TuningStep
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

# Fetch container to use for training
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=config['aws']['region'],
    version=FRAMEWORK_VERSION,
    py_version="py3",
    instance_type=config['input_params']['processing_instance_type'],
)
print(image_uri)

sklearn_estimator = SKLearn(
    entry_point=config['dir_scripts']['training_script'],
    role=role,
    instance_count=1,
    instance_type=config['input_params']['processing_instance_type'],
    framework_version=FRAMEWORK_VERSION,
    base_job_name="rf-scikit",
    hyperparameters={
        "n_estimators": config['training_args']['n_estimators'],
        "max_depth": config['training_args']['max_depth'],  
        "features": config['training_params']['training_features'],
        "target": config['training_params']['training_target'],
    },
    tags=config['pipeline_tags']['sklearn_estimator_tags']
)

## objective metric name
metric_definitions = [{'Name': 'validation:auc', 'Regex': 'auc (\S+)'}]

objective_metric_name = "validation:auc"

# Create Hyperparameter tuner object. Ranges from https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html
rf_tuner = HyperparameterTuner(
                estimator=sklearn_estimator,
                objective_metric_name=objective_metric_name,
                hyperparameter_ranges={
                    "n_estimators": IntegerParameter(10, 150),
                    "max_depth": IntegerParameter(3, 20),
                    "min_samples_split": IntegerParameter(2, 10),
                    "max_features": CategoricalParameter(["sqrt", "log2"])
                },
                max_jobs=config['input_params']['maxium_training_jobs'], ## reducing this for testing purposes
                metric_definitions=metric_definitions,
                max_parallel_jobs=config['input_params']['maxium_parallel_training_jobs'], ## reducing this for testing purposes
)


step_tuning = TuningStep(
    name="Train-And-Tune-Model",
    tuner=rf_tuner,
    inputs={
        "train": TrainingInput(
            s3_data=step_preprocess_data.properties.ProcessingOutputConfig.Outputs[
                "train" ## refer to this
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "test": TrainingInput(
        s3_data=step_preprocess_data.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
        content_type="text/csv",
        ),
    },
)

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3


## Evaluate model step
---

When a model is trained, it's common to evaluate the model on unseen data before registering it with the model registry. This ensures the model registry isn't cluttered with poorly performing model versions. The purpose of the model evaluation step is to check that the trained and tuned model has an accuracy level above a configurable threshold and only then register the model with the model registry (from where it can be subsequently approved and deployed). If the model accuracy does not meet a configured threshold then the pipeline fails and the model is not registered with the model registry.

In [12]:
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile

# Create ScriptProcessor object.
# The object contains information about what container to use, what instance type etc.
evaluate_model_processor = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    role=role,
)

# Create a PropertyFile
# A PropertyFile is used to be able to reference outputs from a processing step, for instance to use in a condition step.
# For more information, visit https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html
evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)


step_evaluate_model = ProcessingStep(
    name="Evaluate-Model",
    processor=evaluate_model_processor,
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=session_bucket),
            destination="/opt/ml/processing/model",
            input_name="model.tar.gz" 
        ),
        ProcessingInput(
            source=step_preprocess_data.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
            input_name="test.csv" 
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation",
            source="/opt/ml/processing/evaluation",
            destination=Join(
                on="/",
                values=[
                    "s3://{}".format(session_bucket),
                    prefix,
                    ExecutionVariables.PIPELINE_EXECUTION_ID,
                    "evaluation",
                ]
            )
        )
    ],
    code = config['dir_scripts']['evaluation_script'],
    property_files=[evaluation_report],
    job_arguments=[
        "--target", target_parameter,
        "--features", feature_parameter,
    ]
)

<a id='register'></a>

## Register model step
If the trained model meets the model performance requirements, a new model version is registered with the model registry for further analysis. To attach model metrics to the model version, create a [ModelMetrics](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html) object using the evaluation report created in the evaluation step. Then, create the RegisterModel step.

The model is registered with the model Registry with approval status set to PendingManualApproval, this means the model cannot be deployed on a SageMaker Endpoint unless its status in the registry is changed to Approved manually via the SageMaker console, programmatically or through a Lambda function.

In [13]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel

# Create ModelMetrics object using the evaluation report from the evaluation step
# A ModelMetrics object contains metrics captured from a model.
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on="/",
            values=[
                step_evaluate_model.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
                    "S3Uri"
                ],
                "evaluation.json",
            ],
        ),
        content_type="application/json",
    )
)



# Crete a RegisterModel step, which registers the model with SageMaker Model Registry.
step_register_model = RegisterModel(
    name="Register-Model",
    estimator=sklearn_estimator,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=session_bucket),
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge", "ml.m5.large"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_registry_package,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
    tags=config['pipeline_tags']['register_model_tags']
    
)

[2024-02-22 18:32:26,201] p145 {utilities.py:465} WARNING - Popping out 'ProcessingJobName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDefinitionConfig to persist this field in the pipeline definition if desired.


<a id='condition'></a>

## Accuracy condition step
Adding conditions to the pipeline is done with a ConditionStep.
In this case, we only want to register the new model version with the model registry if the new model meets an accuracy condition.

In [14]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join

step_fail = FailStep(
    name="AccuracyThresholdFailed",
    error_message=Join(on=" ", values=["Execution failed due to Accuracy <", accuracy_condition_threshold]),
)

In [15]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

# Create accuracy condition to ensure the model meets performance requirements.
# Models with a test accuracy lower than the condition will not be registered with the model registry.
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_evaluate_model.name,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.accuracy.value",
    ),
    right=accuracy_condition_threshold,
)

# Create a SageMaker Pipelines ConditionStep, using the condition above.
# Enter the steps to perform if the condition returns True / False.
step_cond = ConditionStep(
    name="Accuracy-Condition",
    conditions=[cond_gte],
    if_steps=[step_register_model],
    else_steps=[step_fail], ## if this fails - add a step here (from the quip)
)

<a id='orchestrate'></a>

## Pipeline Creation: Orchestrate all steps

Now that all pipeline steps are created, a pipeline is created.

In [16]:
from sagemaker.workflow.pipeline import Pipeline

# Create a SageMaker Pipeline.
# Each parameter for the pipeline must be set as a parameter explicitly when the pipeline is created.
# Also pass in each of the steps created above.
# Note that the order of execution is determined from each step's dependencies on other steps,
# not on the order they are passed in below.
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        training_instance_type,
        accuracy_condition_threshold,
        model_registry_package,
        max_parallel_training_jobs,
        max_training_jobs,
        host_parameter,
        port_parameter,
        user_parameter,
        target_parameter, 
        feature_parameter,
        model_approval_status,
        
    ],
    steps=[
            step_preprocess_data, 
            step_tuning, 
            step_evaluate_model, 
            step_cond],
)

In [17]:
# Submit pipeline
pipeline_upsert_tags = config['pipeline_tags']['pipeline_upsert_tags']
print(pipeline_upsert_tags)
pipeline.upsert(role_arn=role, tags=pipeline_upsert_tags)

[{'Key': 'project-execution', 'Value': 'start-pipeline'}]


[2024-02-22 18:32:26,686] p145 {utilities.py:465} WARNING - Popping out 'ProcessingJobName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDefinitionConfig to persist this field in the pipeline definition if desired.
[2024-02-22 18:32:27,048] p145 {estimator.py:1846} WARNING - No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
[2024-02-22 18:32:27,049] p145 {estimator.py:1846} WARNING - No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
[2024-02-22 18:32:27,050] p145 {utilities.py:465} WARNING - Popping out 'HyperParameterTuningJobName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDefinitionConfig to persist this field in the pipeline definition if desired.
[2024-02

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:597703351594:pipeline/mlops-pipeline-presto',
 'ResponseMetadata': {'RequestId': 'd8086def-277a-4f0f-9702-10d86ce48770',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd8086def-277a-4f0f-9702-10d86ce48770',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '89',
   'date': 'Thu, 22 Feb 2024 18:32:27 GMT'},
  'RetryAttempts': 0}}

## Start pipeline with different parameters.
Now that the pipeline is created, it can be started with custom parameters making the pipeline agnostic to who is triggering it, but also to the scripts and data used. The pipeline can be started using the CLI, the SageMaker Studio UI or the SDK and below there is a screenshot of what it looks like in the SageMaker Studio UI.

#### Starting the pipeline with the SDK
In the examples below, the pipeline is triggered for two machine learning problems, each with different preprocessing scripts and model registry. Each machine learning problem is run with two different sets of parameters.

In [18]:
# Start pipeline with credit data and preprocessing script
execution = pipeline.start(
                execution_display_name=config['pipeline_parameters']['execution_display_name'],
                parameters=dict(
                AccuracyConditionThreshold=config['input_params']['accuracy_condition_threshold'],
                MaxiumParallelTrainingJobs=config['input_params']['maxium_parallel_training_jobs'],
                MaxiumTrainingJobs=config['input_params']['maxium_training_jobs'],
                ModelGroup=order_model_group,
            ),
        )

In [19]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:597703351594:pipeline/mlops-pipeline-presto',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:597703351594:pipeline/mlops-pipeline-presto/execution/8h70jinutn7y',
 'PipelineExecutionDisplayName': 'executeMLOPs-pipeline',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2024, 2, 22, 18, 32, 28, 661000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 2, 22, 18, 32, 28, 661000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:597703351594:user-profile/d-wcu4zlgb8tlc/default-20230918t115140',
  'UserProfileName': 'default-20230918t115140',
  'DomainId': 'd-wcu4zlgb8tlc'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:597703351594:user-profile/d-wcu4zlgb8tlc/default-20230918t115140',
  'UserProfileName': 'default-20230918t115140',
  'DomainId': 'd-wcu4zlgb8tlc'},
 'ResponseMetadata': {'RequestId': '36ace583-a685-40ac-8f99-6a713e553cce',
  'HTTPSt

In [20]:
execution.wait()

In [21]:
execution.list_steps()

[{'StepName': 'Register-Model-RegisterModel',
  'StartTime': datetime.datetime(2024, 2, 22, 18, 45, 54, 833000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 2, 22, 18, 45, 55, 745000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:597703351594:model-package/mlops-presto-model-registry/7'}}},
 {'StepName': 'Accuracy-Condition',
  'StartTime': datetime.datetime(2024, 2, 22, 18, 45, 51, 386000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 2, 22, 18, 45, 54, 75000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata': {'Condition': {'Outcome': 'True'}}},
 {'StepName': 'Evaluate-Model',
  'StartTime': datetime.datetime(2024, 2, 22, 18, 41, 6, 413000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 2, 22, 18, 45, 50, 783000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sag

#### Now that the model is registered, get access to the registered model manually on the sagemaker studio model registry console, or programmatically in the next notebook, approve it and run the second portion of this solution: Batch Transform Step

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker-pipeline-parameterization|parameterized-pipeline.ipynb)
